In [38]:
import numpy as np
import typing
class CNF(object):
    def __init__(self, clauses: np.ndarray):
        """
        Clauses is a two-dimensional array with the following semantics for
        `val = clauses[i, j]`:
            * If val == 0, that space is empty.
            * If val > 0, then the 1-indexed variable val occurs in clause i, unnegated.
            * if val < 0, then the 1-indexed variable val occurs in clause i, negated.
        We assume that clauses[i] consists of some number of nonzero elements,
        followed by zero.
        """
        self.clauses = clauses
        num_variables = np.abs(clauses).max()
        self.num_variables = num_variables

        # Start off assuming every disjunct of every clause is True.
        # If any row of one of the sheets of this array is ever all Falses,
        # then there is a contradiction. The 0th sheet records our current
        # view of known assignments, and subsequent sheets allow scratch work
        # in our backtracking.
        self.positive = np.zeros((self.num_variables, ) + self.clauses.shape,
                                 dtype=bool)
        self.positive[0] = (self.clauses != 0)
        
        # Start off assuming every disjunct of every clause is False.
        # If each row of this array contains a nonzero entry, then
        # we have found a satisfying assignment.
        self.negative = np.zeros((self.num_variables, ) + self.clauses.shape,
                                 dtype=bool)
        
        # Keys are tuples of assignments, sorted by absolute value.
        # For example (-1, 2, 3, -6). `values` a 3-dimensional array
        # where `masks[assignment][0]` is the mask to do bitwise AND
        # with self.positive to get the value after doing the updates
        # in `assignments`. 
        
        self.assignments = set()
        
        self.masks = {}
        # compute masks for all assignments of a single variable.
        for variable in range(1, self.num_variables + 1):
            for assignment in [-1, 1]:
                mask = np.ndarray((2, ) + self.clauses.shape, dtype=bool)
                positive_mask = self.positive[0] & (self.clauses != -assignment * variable)
                if (~positive_mask.any(axis=1)).any():
                    self.assign(-assignment * variable)
                    break
                mask[0] = positive_mask
                negative_mask = (self.clauses == assignment * variable)
                mask[1] = negative_mask
                self.masks[(variable * assignment, )] = mask

    def solve_naive(self):
        """
        Try all possible combinations in a breadth-first search.
        
        Returns a satisfying solution if one exists, otherwise returns None.
        """
        raise NotImplementedError()
                
    def is_satisified(self, depth=0):
        return self.negative[depth].any(axis=1).all()

    def is_inconsistent(self, depth=0):
        return (~self.positive[depth].any(axis=1)).any()

    @classmethod
    def load_dimacs(cls, s: str):
        """Loads a boolean expression from a string.

        Based off sympy.logic.utilities.dimacs
        https://github.com/sympy/sympy/blob/57fcd5a941d7c47106bd63fd7b3d79ac032b636b/sympy/logic/utilities/dimacs.py
        """
        import re
        clauses = []
        pComment = re.compile(r'c.*')
        pStats = re.compile(r'p\s*cnf\s*(\d*)\s*(\d*)')
        lines = [line.strip() for line in s.splitlines()
                 if not pComment.match(line) and not pStats.match(line)]
        clauses = []
        for line in lines:
            nums = [int(num) for num in line.rstrip('\n').split(' ')
                    if num not in {'', '0'}]
            if nums:
                clauses.append(nums)
        a = np.zeros((len(clauses), max(map(len, clauses))))
        import itertools
        return cls(np.array(list(itertools.zip_longest(*clauses,fillvalue=0)), dtype=np.int16).T)

    def evaluate(self) -> np.ndarray:
        """
        Returns a new clause array with self.assignments applied.
        """
        # Remove satisified clauses:
        unsatisfied_rows = (~self.negative[0].any(axis=1)).nonzero()
        new_clauses = self.clauses[unsatisfied_rows]

        # Remove contradictory atoms:
        for assignment in self.assignments:
            new_clauses *= (new_clauses != -assignment)

        # Shrink the array to remove zero columns:
        new_clauses = new_clauses[np.unravel_index(np.argsort(-np.abs(new_clauses), axis=1), new_clauses.shape)]
        t = new_clauses.T
        new_clauses = t[~(t==0).all(1)].T
        return new_clauses

    def assign(self, *assignments: typing.List[int]):
        assignments = set(assignments)
        todo = assignments.copy()
        while todo:
            assignment = todo.pop()
            self.assignments.add(assignment)
            if (-assignment, ) in self.masks:
                del self.masks[(-assignment, )]
            if (assignment, ) not in self.masks:
                continue
            assignment_mask = self.masks.pop((assignment, ))
            self.positive[0] &= assignment_mask[0]
            self.negative[0] |= assignment_mask[1]
            for t, submask in list(self.masks.items()):
                if -assignment in t:
                    del self.masks[t]
                elif assignment in t:
                    new_t = tuple(a for a in t if a != assignment)
                    self.masks[new_t] = submask
                    del self.masks[t]
                else:
                    submask[0] &= assignment_mask[0]
                    submask[1] |= assignment_mask[1]
                    if (~(submask[0].any(axis=1))).any():
                        # This assignment has become inconsistent, so remove it.
                        del self.masks[t]
                        if len(t) == 1:
                            if -t[0] in self.assignments:
                                continue
                            elif -t[0] in assignments:
                                continue
                            else:
                                print('Deduced %s from %s.' % (-t[0], assignment))
                                assignments.add(-t[0])
                                todo.add(-t[0])
                        elif len(t) == 2:
                            a1, a2 = t
                            # Since (a1 & a2) is inconsistent, then we can deduce the following:
                            # a1 -> ~a2 and a2 -> ~a1. Update the masks accordingly if applicable.
                            if (a1, ) in self.masks and (-a2, ) in self.masks:
                                print('Deduced %s->%s from %s' % (a1, -a2, assignment))
                                self.masks[(a1, )][0] &= self.masks[(-a2, )][0]
                                self.masks[(a1, )][1] |= self.masks[(-a2, )][1]
                            if (-a1, ) in self.masks and (a2, ) in self.masks:
                                print('Deduced %s->%s from %s' % (a2, -a1, assignment))
                                self.masks[(a2, )][0] &= self.masks[(-a1, )][0]
                                self.masks[(a2, )][1] |= self.masks[(-a1, )][1]
                                


In [39]:
factor_35_dimacs = '''\
p cnf 6 6 
c Factors encoded in variables 1-3 and 4-6
c Target number: 35
-2 -5 0
1 0
3 0
4 0
6 0
2 5 0
'''
problem = CNF.load_dimacs(factor_35_dimacs)
print(problem.clauses)
print(problem.assignments)
problem.assign(2)
print(problem.assignments)

[[-2 -5]
 [ 1  0]
 [ 3  0]
 [ 4  0]
 [ 6  0]
 [ 2  5]]
{1, 3, 4, 6}
Deduced -5 from 2.
{1, 2, 3, 4, 6, -5}


In [1]:
example.clauses

NameError: name 'example' is not defined